In [1]:
from collections import Counter

import numpy as np
import pandas as pd

import itertools

# grid search hyperparameters for lasso regression
from numpy import arange
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import os

In [2]:
# Change working directory to be in the project folder and print out what is in the data folder
os.chdir('/content/drive/MyDrive/Free time (Aaron Presser): Poverty prediction project (02 14 2022)/')
os.listdir('Output_csv_files')

['food_access_df.csv',
 'food_env_df.csv',
 'merged_df.csv',
 'fastfood_df.csv',
 'fastfood_df_dup.csv',
 'walmart_df.csv',
 'merged_df_10.csv']

In [84]:
path_to_df = 'Output_csv_files/merged_df_10.csv'
df = pd.read_csv(path_to_df).drop(columns=['Unnamed: 0'])
df

,LATracts10,Restaurant Name,Zip Code,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,0,Black Cow Burger Bar,1376,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,631.0,3798.0,74.0,29.0,1.0,18.0,293.0,346.0,268.0,449.0
1,0,Black Cow Burger Bar,1376,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,715.0,4025.0,27.0,38.0,1.0,7.0,126.0,92.0,122.0,311.0
2,0,Black Cow Burger Bar,1380,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,459.0,3319.0,21.0,17.0,0.0,16.0,55.0,60.0,70.0,213.0
3,0,Black Cow Burger Bar,1420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,330.0,2770.0,149.0,104.0,0.0,14.0,187.0,396.0,15.0,55.0
4,0,Black Cow Burger Bar,1420,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,306.0,1844.0,41.0,126.0,0.0,6.0,132.0,271.0,24.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129506,0,Pizza Mill,99926,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,133.0,145.0,6.0,2.0,10.0,1210.0,87.0,26.0,59.0,164.0
129507,1,Pizza Mill,99927,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,181.0,1217.0,5.0,6.0,7.0,351.0,99.0,29.0,111.0,154.0
129508,0,Pizza Mill,99928,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,351.0,3013.0,9.0,33.0,4.0,186.0,239.0,103.0,58.0,116.0
129509,0,Pizza Mill,99928,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,351.0,3013.0,9.0,33.0,4.0,186.0,239.0,103.0,58.0,116.0


In [85]:
restaurant_name = ",".join(df['Restaurant Name']).split(",")
df['Restaurant Name'].str.get_dummies(",")

,Burgers And Bowls,Inc.,Salads More,1 Gyro Shoppe,143 Street Tacos,2 Bits Cafe,5 Star Abq,5th Street Diner,900 Degrees Pizza,919 Dine,...,Zappo's Pizza,Zaxby's,Zeeks Pizza,Zel's Roast Beef All,Zesto of West Columbia,Zippers Restaurant Lounge,Zippy's,Zips It's About Time Drive In,Zorba's Gyro On A Spit,sweetgreen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
df.drop(columns=['Restaurant Name', 'Zip Code'], inplace=True)
df.drop(index=df[df['Location'].apply(lambda x: x.startswith('Walmart '))].index, inplace=True)

In [ ]:
counted = Counter(",".join(dataframe['Categories']).split(","))
{k: v for k, v in sorted(counted.items(), key=lambda item: item[1], reverse=True)}

NameError: ignored

In [87]:
def process_contains(df, column, contains, replace):
  if contains is None:
    contains = ['Cater', 'Delic', 'Hot dog', 'Mexican', 'Sandwich', 'Coffee', 'Veg', 'Pizza']
  else: contains = set(df[f"{contains}"])
  
  if replace is None:
    replace = ['Catering', 'Deli', 'Hot dogs', 'Mexican', 'Sandwich', 'Coffee', 'Vegetarian', 'Pizza']
  else: replace = set(df[f"{replace}"])
  
  if column is None:
    column = 'categories'

  variable_dict = {}
  for i, j, k in zip(contains, replace, range(len(contains))):
    boolean_mask = df[f"{column}"].str.contains(i)
    variable_dict[f'{j}'] = boolean_mask
  
  df.drop(columns=[f"{column}"], inplace=True)
  variable_df = pd.DataFrame(variable_dict)
  variable_df.replace({False: 0, True: 1}, inplace=True)

  return df.join(variable_df)

In [88]:
df = process_contains(df, column = 'Location', contains='Location', replace='Location')
# dataframe = process_contains(dataframe, column = 'Car Juice', contains='Car Juice', replace='Car Juice')
df

,LATracts10,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,Pizza,Car Juice,...,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Sam's Club,Walmart,SC,Murphy
0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,18.0,293.0,346.0,268.0,449.0,0,0,1,0
1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,7.0,126.0,92.0,122.0,311.0,0,0,1,0
2,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,16.0,55.0,60.0,70.0,213.0,0,0,1,0
3,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.0,187.0,396.0,15.0,55.0,0,0,1,0
4,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,132.0,271.0,24.0,179.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129506,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,10.0,1210.0,87.0,26.0,59.0,164.0,0,1,0,0
129507,1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,7.0,351.0,99.0,29.0,111.0,154.0,0,1,0,0
129508,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,4.0,186.0,239.0,103.0,58.0,116.0,0,1,0,0
129509,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,4.0,186.0,239.0,103.0,58.0,116.0,0,1,0,0


In [89]:
corr_matrix = df.corr()
corr_matrix = corr_matrix[corr_matrix > 0.93]

for i in range(len(corr_matrix.columns)):
  corr_matrix.iloc[i,i] = np.nan
shit_matrix = corr_matrix.dropna(axis=0, how='all')
df.drop(columns=shit_matrix.index, inplace=True)

In [ ]:
for i, j in zip(df.columns, df.dtypes):
  print(i)
  print(j)
  print()

In [90]:
from sklearn.feature_selection import VarianceThreshold

def variance_threshold_selector(data, threshold=0.1):
    # https://stackoverflow.com/a/39813304/1956309
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [91]:
# shit_df = df
df = variance_threshold_selector(df, threshold=0.02)
df

,LATracts10,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,Pizza,Car Juice,...,TractAsian,TractNHOPI,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Sam's Club,Walmart,SC,Murphy
0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.0,1.0,293.0,346.0,268.0,449.0,0,0,1,0
1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,1.0,126.0,92.0,122.0,311.0,0,0,1,0
2,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,55.0,60.0,70.0,213.0,0,0,1,0
3,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,104.0,0.0,187.0,396.0,15.0,55.0,0,0,1,0
4,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,126.0,0.0,132.0,271.0,24.0,179.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129506,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2.0,10.0,87.0,26.0,59.0,164.0,0,1,0,0
129507,1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,6.0,7.0,99.0,29.0,111.0,154.0,0,1,0,0
129508,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,33.0,4.0,239.0,103.0,58.0,116.0,0,1,0,0
129509,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,33.0,4.0,239.0,103.0,58.0,116.0,0,1,0,0


In [71]:
zero_df = df[df['LATracts10'] == 0]
zero_df_sample = zero_df.sample(frac=0.9)
df.drop(labels=zero_df_sample.index, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [92]:
random_train_subset = df.sample(frac=0.9)
X_train, y_train = random_train_subset[df.columns[1:]], random_train_subset[df.columns[:1]]

random_test_subset = df.drop(labels=X_train.index)
X_test, y_test = random_test_subset[df.columns[1:]], random_test_subset[df.columns[:1]]

df

,LATracts10,Catering,Deli,Hot dogs,Mexican,Sandwich,Coffee,Vegetarian,Pizza,Car Juice,...,TractAsian,TractNHOPI,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Sam's Club,Walmart,SC,Murphy
0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.0,1.0,293.0,346.0,268.0,449.0,0,0,1,0
1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,1.0,126.0,92.0,122.0,311.0,0,0,1,0
2,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,55.0,60.0,70.0,213.0,0,0,1,0
3,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,104.0,0.0,187.0,396.0,15.0,55.0,0,0,1,0
4,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,126.0,0.0,132.0,271.0,24.0,179.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129506,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2.0,10.0,87.0,26.0,59.0,164.0,0,1,0,0
129507,1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,6.0,7.0,99.0,29.0,111.0,154.0,0,1,0,0
129508,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,33.0,4.0,239.0,103.0,58.0,116.0,0,1,0,0
129509,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,33.0,4.0,239.0,103.0,58.0,116.0,0,1,0,0


In [73]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [93]:
def predict_and_score(model, X_test, y_test):
  # performing predictions on the test datdaset
  yhat = model.predict(X_test)
  prediction = list(map(round, yhat))
  
  # comparing original and predicted values of y
  print('Actual values', list(y_test.values))
  print('Predictions :', prediction)


  from sklearn.metrics import (confusion_matrix,
                            accuracy_score)
  
  # confusion matrix
  cm = confusion_matrix(y_test, prediction)
  print ("Confusion Matrix : \n", cm)
  
  # accuracy score of the model
  print('Test accuracy = ', accuracy_score(y_test, prediction, normalize='True'))
  return

In [83]:
predict_and_score(log_reg, X_test, y_test)

Actual values [array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([0]), array([0]), array([0]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), a

In [ ]:
# log_reg = sm.Logit(y_train, X_train).fit()
# log_reg.save("food_desert_model_statsmodels.pickle")
log_reg.pvalues

In [94]:
# df[df.columns[0:1]]

# # data = dataframe.values
# X_train, X_test = df.sample(frac=0.9, random_state=8)[df.columns[2:]], df[~X_train[df.columns[1:]]]
# y_train, y_test
# # y = dataframe.drop(labels=X.index)
# X, y = df[df.columns[1:2]], df[df.columns[2:]]
# define model
model = LogisticRegression(solver='liblinear', class_weight='balanced')
model.fit(X_train, y_train.values.ravel())
params = np.append(model.intercept_, model.coef_)
predictions = model.predict(X_test)
score = model.score(X_test, y_test)
print(f"Score: {score}\n")

predict_and_score(model, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Score: 0.8690310616597126

Actual values [array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])

In [77]:
import pickle

# save the model to disk
filename = 'food_desert_model_sklearn.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model.decision_function(X_test)

(12942,)

In [ ]:
score

0.9690156080976665

In [ ]:
import seaborn as sns

In [ ]:
sns.regplot(x=df[1:].columns, y=df[0].columns, data=df, logistic=True)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
random_train_subset.columns[:1]

Index(['LATracts10'], dtype='object')

In [ ]:
X = random_train_subset[random_train_subset.columns[1:]]
Y = random_train_subset[random_train_subset.columns[:1]]
log_reg = smf.logit(Y, X, data=random_train_subset).fit()

TypeError: ignored

In [ ]:
random_train_subset.columns[1:]

In [ ]:
def jitter(X, scale=0.1):
    return X + np.random.normal(0, scale, X.shape)

def jitter_test(classifier, X, y, scales = np.linspace(0, 0.5, 30), N = 5):
    out = []
    for s in scales:
        avg = 0.0
        for r in range(N):
            avg += metrics.accuracy_score(y, classifier.predict(jitter(X, s)))
        out.append(avg / N)
    return out, scales

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train.values.ravel())
metrics.accuracy_score(y_test, model.predict(X_test + np.random.normal(0, 0.1, X_test.shape)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.969788286199969

In [ ]:
metrics.accuracy_score(y_test, model.predict(jitter(X_test, 0.1)))

0.9690928759078967

In [ ]:
# scores, jitters = jitter_test(model, X_test, y_test)
print(jitters)

[0.         0.01724138 0.03448276 0.05172414 0.06896552 0.0862069
 0.10344828 0.12068966 0.13793103 0.15517241 0.17241379 0.18965517
 0.20689655 0.22413793 0.24137931 0.25862069 0.27586207 0.29310345
 0.31034483 0.32758621 0.34482759 0.36206897 0.37931034 0.39655172
 0.4137931  0.43103448 0.44827586 0.46551724 0.48275862 0.5       ]
